<a href="https://colab.research.google.com/github/rbgiri/Natural-Language-Processing/blob/main/Fine_tune_gemma_LLm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Libraries

In [ ]:
!pip install -U accelerate bitsandbytes transformers huggingface_hub

In [1]:
from huggingface_hub import notebook_login
notebook_login()

# Typing Important Comand

In [3]:
#Import necessary classes for model loading and quantization
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configure model quantization to 4-bit for memory and computation efficiency
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

# Load the tokenizer for the Gemma 7B Italian model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it")

# Load the Gemma 7B Italian model itself, with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it",
                                             quantization_config=quantization_config)

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.11G [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

# Inferencing the model

In [4]:
# Define input text:
input_text = "List the key points about Responsible AI"

# Tokenize the input text:
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate text using the model:
outputs = model.generate(
    **input_ids,  # Pass tokenized input as keyword argument
    max_length=512,  # Limit output length to 512 tokens
)

# Decode the generated text:
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


<bos>List the key points about Responsible AI.

**Key Points of Responsible AI:**

* **Accountability:** Ensuring that AI systems are transparent, explainable, and accountable to human oversight.
* **Fairness:** Mitigating bias and discrimination in AI systems to ensure fairness and equity for all.
* **Reliability:** Maintaining the accuracy, consistency, and robustness of AI systems to ensure reliability and trust.
* **Privacy:** Protecting individual privacy and data rights in AI systems.
* **Explainability:** Making AI systems understandable and explainable to humans, enabling human oversight and decision-making.
* **Human-in-the-Loop:** Emphasizing the importance of human-in-the-loop design, where humans and AI systems work together to enhance decision-making.
* **Ethical Considerations:** Addressing ethical concerns related to AI, such as job displacement and potential misuse.
* **Continuous Improvement:** Promoting ongoing improvement and evolution of AI systems to address emergi

# Response Generation

In [5]:
input_text = "How many eggs can a Whale lay in its lifetime?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids,max_length=512)
print(tokenizer.decode(outputs[0]))

<bos>How many eggs can a Whale lay in its lifetime?

The answer is: Whales do not lay eggs. Whales are mammals and do not lay eggs like fish or birds.<eos>


In [6]:
input_text = "How many smartphones can a human eat ?"
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids,max_length=512)
print(tokenizer.decode(outputs[0]))

<bos>How many smartphones can a human eat ?

The answer is zero. Humans cannot eat smartphones. Smartphones are electronic devices, not food.<eos>


In [7]:
input_text = "I have 3 apples and 2 oranges. I ate 2 organes. How many apples do I have?"
input_ids = tokenizer(input_text, return_tensors="pt").to('cuda')
outputs = model.generate(**input_ids,max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

<bos>I have 3 apples and 2 oranges. I ate 2 organes. How many apples do I have?

The answer is 3 - 2 = 1 apple.<eos>


In [8]:
input_text = "I have 3 apples and 2 oranges. \
I ate 2 oranges. How many apples do I have? \
Think Step by Step. For each step, re-evaluate your answer"
input_ids = tokenizer(input_text, return_tensors="pt").to('cuda')
outputs = model.generate(**input_ids,max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

<bos>I have 3 apples and 2 oranges. I ate 2 oranges. How many apples do I have? Think Step by Step. For each step, re-evaluate your answer to see if it is correct.

**Step 1:** Identify the number of apples and oranges I have.

**Answer:** I have 3 apples and 2 oranges.

**Re-evaluation:** My answer is correct.

**Step 2:** Subtract the number of oranges I ate from the number of oranges I have.

**Answer:** I have 2 oranges left.

**Re-evaluation:** My answer is incorrect. I should have subtracted 2 from 2, not from 3.

**Step 3:** Subtract the number of oranges I have left from the number of apples I have.

**Answer:** I have 3 apples left.

**Re-evaluation:** My answer is correct.

Therefore, I have 3 apples left.<eos>


In [9]:
input_text = "Write a hello world program"
input_ids = tokenizer(input_text, return_tensors="pt").to('cuda')
outputs = model.generate(**input_ids,max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

<bos>Write a hello world program in Python.

```python
print("Hello, world!")
```

**Output:**

```
Hello, world!
```

**Explanation:**

* The `print` function is used to print the message to the console.
* The string "Hello, world!" is passed as an argument to the `print` function.
* The output of the program is printed to the console as "Hello, world!".<eos>


In [10]:
input_text = "covert the said comment in english only 'main account open karane jaa rha, kal aunga'"
input_ids = tokenizer(input_text, return_tensors="pt").to('cuda')
outputs = model.generate(**input_ids,max_new_tokens=512)
print(tokenizer.decode(outputs[0]))

<bos>covert the said comment in english only 'main account open karane jaa rha, kal aunga'

Sure, here is the translation in English:

"Main account open karane jaa rha, kal aunga."

Translation:

"The main account is open, come on in and join us."<eos>
